In [3]:
import json
import re

import requests
import scrapy

In [4]:
url ="http://goheels.com/roster.aspx?path=mbball"
resp = requests.get(url, headers={'User-Agent': 'UNC Journo Class'},)

In [5]:
body_bytes = resp.content

In [6]:
body_str = body_bytes.decode('utf-8')

In [7]:
body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide mbball sport mbball"> \r\n    <head><script>window.client_hostname = "goheels.com";window.server_name = "866411-LB16";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"mbball","sport_name_custom":"mbball","site":"unc","content_id":null,"sect":"mbaskbl"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636518877046678376"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><link rel="canonical" href="http://goheels.com/roster.aspx?roster=476&amp;path=mbball"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-icon" href="/image

In [8]:
sel = scrapy.Selector(text=body_str)

In [9]:
table = sel.css('.sidearm-table')[0]

In [10]:
table

<Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' sidearm-table ')]" data='<table class="sidearm-table sidearm-tabl'>

In [11]:
table.css('tr') # Get all the tr elements within the table
table.css('tr')[0] # Get the first tr element within the table using the pythonic index
table.css('tr')[0].css('th') # Get all the th elements from within the first tr
table.css('tr:nth-child(1) th')

[<Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' 

In [12]:
table.css('tr:nth-child(1) th')

[<Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' 

In [13]:
table.css('tr')[0].css('th').xpath('string()').extract()

['No.', 'Name', 'Pos.', 'Ht.', 'Wt.', 'Yr.', 'Hometown / High School']

In [14]:
col_names = table.css('tr:nth-child(1) th').xpath('string()').extract()
col_names

['No.', 'Name', 'Pos.', 'Ht.', 'Wt.', 'Yr.', 'Hometown / High School']

In [15]:
other_rows = table.css('tr:not(:nth-child(1))')
other_rows

[<Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0)]' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr[not(count(preceding-sibling::*) = 0

In [16]:
# my starting point for just getting the information from the rows
by_player_num = {}
for row in other_rows:
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_name_data = []
    
    by_player_num[player_num] = player_name, player_position

In [17]:
by_player_num

{'1': ('Theo Pinson', 'F/G'),
 '11': ('Shea Rush', 'F'),
 '13': ('Cameron Johnson', 'G'),
 '14': ('Kane Ma', 'G'),
 '15': ('Garrison Brooks', 'F'),
 '2': ('Joel Berry II', 'G'),
 '21': ('Sterling Manley', 'F'),
 '22': ('Walker Miller', 'F'),
 '24': ('Kenny Williams', 'G'),
 '25': ('Aaron Rohlman', 'F'),
 '3': ('Andrew Platek', 'G'),
 '30': ('K.J. Smith', 'G'),
 '32': ('Luke Maye', 'F'),
 '4': ('Brandon Robinson', 'G'),
 '42': ('Brandon Huffman', 'F'),
 '5': ('Jalek Felton', 'G')}

In [18]:
#populate all the data to the object organized by number
by_player_num = {}
player_data = []
for row in other_rows:
    
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    # get the second cell, then the link, then just the href
    link = row.css('td')[1]
    link = link.css('a')
    player_href= link.xpath('@href').extract()
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_data = {
        "name": player_name,
        "position": player_position,
        "href": player_href[0],
    }
    
    
    
#     player_data.append({
#             "name": player_name,
#             "position": player_position,
#             "href": player_href[0],
#         })
    
    by_player_num[player_num] = player_data

In [19]:
by_player_num

{'1': {'href': '/roster.aspx?rp_id=13515',
  'name': 'Theo Pinson',
  'position': 'F/G'},
 '11': {'href': '/roster.aspx?rp_id=13518',
  'name': 'Shea Rush',
  'position': 'F'},
 '13': {'href': '/roster.aspx?rp_id=13525',
  'name': 'Cameron Johnson',
  'position': 'G'},
 '14': {'href': '/roster.aspx?rp_id=14170',
  'name': 'Kane Ma',
  'position': 'G'},
 '15': {'href': '/roster.aspx?rp_id=13522',
  'name': 'Garrison Brooks',
  'position': 'F'},
 '2': {'href': '/roster.aspx?rp_id=13508',
  'name': 'Joel Berry II',
  'position': 'G'},
 '21': {'href': '/roster.aspx?rp_id=13526',
  'name': 'Sterling Manley',
  'position': 'F'},
 '22': {'href': '/roster.aspx?rp_id=13527',
  'name': 'Walker Miller',
  'position': 'F'},
 '24': {'href': '/roster.aspx?rp_id=13520',
  'name': 'Kenny Williams',
  'position': 'G'},
 '25': {'href': '/roster.aspx?rp_id=13517',
  'name': 'Aaron Rohlman',
  'position': 'F'},
 '3': {'href': '/roster.aspx?rp_id=13528',
  'name': 'Andrew Platek',
  'position': 'G'},
 '30'

In [26]:
for player in list(by_player_num.values()):
    new_url = 'http://goheels.com' + player["href"]
    resp = requests.get(new_url, headers={'User-Agent': 'UNC Journo Class'},)
    content = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=content)
    #selects the image url
    player_image = sel.css('.sidearm-roster-player-image img').xpath("@src").extract()[0]
    # selects the biography
    player_bio = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    #player_data = by_player_num[player_num]
    
    player_id = re.search("rp_id=(\d+)", player["href"]).group(1)
    stats_url = 'http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id='+player_id+'&path=mbball&year=2017&player_id='+player_id
    stats_resp = requests.get(stats_url, headers={'User-Agent': 'UNC Journo Class'},)
    stats_content = stats_resp.content.decode('utf-8')
    stats_sel = scrapy.Selector(text=stats_content)
    
    html = json.loads(stats_sel.content.decode("utf-8"))["current_stats"]
    stats_sel = scrapy.Selector(text=html)
    
    player_stats = stats_sel.css('.sidearm-table').xpath('string()').extract()

    player["image"] = player_image
    player["bio"] = player_bio
    player["stats"] = player_stats
        
    print(player_id)
    print(player_stats)
#     by_player_num[player_num] = player_data

AttributeError: 'Selector' object has no attribute 'content'

In [24]:
by_player_num

{'1': {'bio': "\r\n                        Biography\r\n                                                    \r\n                            CAREER\r\nOne of 12 Tar Heels to play in two national championship games • Played in 85 games with 21 starts • Has 377 points, 296 rebounds, 34 three-pointers, 229 assists and an assist/error ratio of 2.4 • Scored in double figures 10 times with highs of 13 vs. Clemson on 12/30/15 and Louisville on 2/22/17 • Has led UNC in assists 21 times.\r\n\r\n\r\nJUNOR SEASON (2016-17)\r\nAveraged 7.0 points, 8.5 rebounds and 3.5 assists in the wins over Oregon and Gonzaga in the Final Four • The 17 rebounds are the most he ever had in consecutive college games • Averaged 6.1 points, 4.6 rebounds and 3.7 assists in 21 games • Tar Heels went 18-3 in the 21 games he played • Missed 19 games over two different spans (sat out the first 16 games due to injury, returned to play in six, then missed three more games due to injury) • Had 77 assists and 31 turnovers\xa0